After filtering the discourses based on the keywords we choose, let's clean them up. 

There are distinct patterns at the beginning of each discourse that separate the names of the representatives, the political party, whether the speech was given or was submitted. We need to remove these sentences, otherwise they will mess the final results.

In [ ]:
import pandas as pd
import re
import spacy
import numpy as np
import nltk
import string
import time
from nltk.tokenize import RegexpTokenizer
from collections import Counter

In [ ]:
%%time
nlp = spacy.load("pt_core_news_lg")
print('spaCy Version: %s' % (spacy.__version__))

spaCy Version: 3.0.6
Wall time: 23.3 s


In [ ]:
df = pd.read_csv('amazon_discourses.tsv', sep='\t', encoding="utf-8")

In [ ]:
df

,date,session,phase,discourse_link,speaker,party,coalition,state,region,original_discourse
0,2000-01-11,195.2.51.O,short address,https://www.camara.leg.br/internet/SitaqWeb/Te...,nilton capixaba,ptb,0,ro,north,O SR. NILTON CAPIXABA (PTB-RO. Pronuncia o seg...
1,2000-01-11,195.2.51.O,parliamentarian communications,https://www.camara.leg.br/internet/SitaqWeb/Te...,sérgio novais,psb,0,ce,northeast,O SR. SÉRGIO NOVAIS (Bloco/PSB-CE. Como Líder....
2,2000-01-11,195.2.51.O,short address,https://www.camara.leg.br/internet/SitaqWeb/Te...,marcos afonso,pt,0,ac,north,O SR. MARCOS AFONSO (PT-AC. Pronuncia o seguin...
3,2000-01-11,195.2.51.O,short address,https://www.camara.leg.br/internet/SitaqWeb/Te...,expedito júnior,dem,1,ro,north,O SR . EXPEDITO JÚNIOR (PFL-RO. Pronuncia o se...
4,2000-03-11,196.2.51.O,long address,https://www.camara.leg.br/internet/SitaqWeb/Te...,silas câmara,ptb,0,am,north,O SR. SILAS CÂMARA (PTB-AM. Sem revisão do ora...
...,...,...,...,...,...,...,...,...,...,...
14597,2021-12-15,162.2021,agenda,https://www.camara.leg.br/internet/sitaqweb/Te...,cleber verde,republicanos,0,ma,northeast,O SR. CLEBER VERDE (REPUBLICANOS - MA. Pela or...
14598,2021-12-16,163.2021,brief communications,https://www.camara.leg.br/internet/sitaqweb/Te...,josé ricardo,pt,0,am,north,O SR. JOSÉ RICARDO (PT - AM. Sem revisão do or...
14599,2021-12-16,163.2021,brief communications,https://www.camara.leg.br/internet/sitaqweb/Te...,sidney leite,psd,0,am,north,O SR. SIDNEY LEITE (PSD - AM. Sem revisão do o...
14600,2021-12-16,163.2021,brief communications,https://www.camara.leg.br/internet/sitaqweb/Te...,bohn gass,pt,0,rs,south,O SR. BOHN GASS (PT - RS. Sem revisão do orado...


In [ ]:
# we first need to make sure some patterns exists:
df['original_discourse'] = df['original_discourse'].str.replace('–', '-')
df['original_discourse'] = df['original_discourse'].str.replace('—', '-')
df['original_discourse'] = df['original_discourse'].str.replace(" – ", ' - ')

df['original_discourse'] = df['original_discourse'].str.replace("\.\) -", "\.\) - ")
df['original_discourse'] = df['original_discourse'].str.replace("\.\) ", "\.\) - ")
df['original_discourse'] = df['original_discourse'].str.replace("\.\)-", "\.\) - ")
df['original_discourse'] = df['original_discourse'].str.replace("\.\)\. - ", "\.\) - ")

df['original_discourse'] = df['original_discourse'].str.replace("\.\)S", "\.\) - S")

In [ ]:
# create a copy
df1 = df.copy()

In [ ]:
size = df.shape[0]
print(size)

14602


**1st approach:** sentences regularly start with an ") - ", so we'll split the discourses into two parts and hold the last one which actually holds the discourse.

In [ ]:
df['clean_discourse'] = df.original_discourse.str.split("orador\.\) - ", n=2, expand=True)[1]

As expected, this approach does not work on the others patterns, giving us some empty discourses, as we can see below.

In [ ]:
# checking the null entries where the split hasn't worked well
null_speeches = df.loc[df.clean_discourse.isna()]
print("There are %i null speeches" % len(null_speeches))

There are 110 null speeches


In [ ]:
print(df.original_discourse[397], "\n\n", 
      df.original_discourse[1464], "\n\n",
      df.original_discourse[1703], "\n\n",
      df.original_discourse[1927], "\n\n",
      df.original_discourse[2430], "\n\n",
      df.original_discourse[2431])

O SR. CONFÚCIO MOURA (PMDB-RO. Pronuncia o seguinte discurso\.\) -  Sr. Presidente, Sras. e Srs. Deputados, grande parte da população brasileira pensa que a atividade parlamentar resume-se às votações e aos discursos no plenário. Ledo engano. A missão de grande parte dos Parlamentares, de sexta a segunda-feira, consiste numa via-sacra de reuniões, visitas, debates e palestras nos Municípios de sua base política.Na semana passada visitei dois Municípios no Estado de Rondônia: Theobroma e Jaru, que ficam a 25 quilômetros de distância um do outro. O primeiro foi distrito do outro por muitos anos.São Municípios irmãos, com problemática semelhante, tendo eles, ainda, a maioria de suas populações localizada na área rural e sendo sede da maior bacia leiteira da Amazônia Ocidental. Enormes são as dificuldades dos Prefeitos para a manutenção de suas estradas, principalmente em Rondônia, que tem um dos mais elevados índices de chuva do País, para deixá-las em condições de tráfego, para o transpo

Thus, we remove these empty discourses and follow a 2nd approach to the copied dataset.

In [ ]:
df.dropna(inplace=True)

In [ ]:
# assert that the number of rows decreased
assert df.shape[0] < size

Now, it's time to process the 110 discourses left not covered by the previous approach.

In [ ]:
df1 = df1.loc[null_speeches.index]

In [ ]:
assert df1.shape[0] == 110

**2nd approach:** now we split the data using the pattern of any word in capital letter, "-" followed by "Sr." or "Sra." and retrieve the last string obtained in the split:

In [ ]:
df1['clean_discourse'] = df1.original_discourse.str.split("[A-Z]+ - Sr\.|Sra\.", n=1, expand=True)[1]

We still have inconsistent entries because there still is data with a different pattern.

In [ ]:
null_speeches_1 = df1.loc[df1.clean_discourse.isna()]
print("There are %i null speeches" % len(null_speeches_1))

There are 53 null speeches


Some entries do not fit the standards. We will have to modify them manually.

In [ ]:
print(df1.original_discourse[12220])

O SR. DEPUTADO HILDO ROCHA - Obrigado, Deputado Zé Geraldo, que preside esta Comissão Geral sobre os Correios e Telégrafos. Sras. e Srs. Deputados, convidados, Sr. Presidente dos Correios, demais representantes desta instituição, as dificuldades operacionais dos Correios no momento arrastam para a lama essa instituição que já esteve no topo como uma das mais confiáveis e respeitadas do nosso País. Nos últimos 10 anos, nós temos observado um processo de deterioração dessa importante instituição, um dos maiores patrimônios da sociedade brasileira. Sem dúvida nenhuma, o que ocorreu com os Correios não foi outra coisa senão a má gestão e os assaltos acontecidos na empresa. Isso já foi devidamente comprovado nas investigações sobre o mensalão e outros atos de corrupção que já foram apresentados a todo o povo brasileiro. É inaceitável que para debelar a crise dos Correios quem pague a pena sejam os usuários dos Correios, que é o povo brasileiro, e os funcionários dessa instituição. Onde há u

In [ ]:
df1.clean_discourse[12220] = "Obrigado, Deputado Zé Geraldo, que preside esta Comissão Geral sobre os Correios e Telégrafos. Sras. e Srs. Deputados, convidados, Sr. Presidente dos Correios, demais representantes desta instituição, as dificuldades operacionais dos Correios no momento arrastam para a lama essa instituição que já esteve no topo como uma das mais confiáveis e respeitadas do nosso País. Nos últimos 10 anos, nós temos observado um processo de deterioração dessa importante instituição, um dos maiores patrimônios da sociedade brasileira. Sem dúvida nenhuma, o que ocorreu com os Correios não foi outra coisa senão a má gestão e os assaltos acontecidos na empresa. Isso já foi devidamente comprovado nas investigações sobre o mensalão e outros atos de corrupção que já foram apresentados a todo o povo brasileiro. É inaceitável que para debelar a crise dos Correios quem pague a pena sejam os usuários dos Correios, que é o povo brasileiro, e os funcionários dessa instituição. Onde há um Município há uma agência dos Correios. É a agência dos Correios que conecta o cidadão brasileiro que mora nos lugares mais afastados do Brasil. Na Região Amazônica, no Maranhão, em Municípios e povoados de difícil acesso, o cidadão tem assegurado pelo Estado o seu direito de receber correspondência, enviar telegrama, encaminhar encomenda. Se essa instituição for privatizada, logicamente essas comunidades deixarão de receber esses serviços. Nós pudemos observar isso com o sistema de telefonia móvel. Nas localidades mais afastadas do nosso País, mesmo com toda a força que os Governos têm feito para levar banda larga e melhorar o acesso à Internet, esses serviços não têm se concretizado. O mesmo acontecerá com os serviços prestados pelos Correios, porque uma empresa quer ter lucro. Uma empresas não quer se privar do lucro, e está certa. A empresa tem que se manter, tem que ter lucro mesmo. Ela não vai querer oferecer serviços nos Municípios mais distantes do nosso País. Por isso, é necessário que a mudança seja feita não por meio da transmissão desse patrimônio para a iniciativa privada, mas por meio da melhoria na gestão dessa empresa. Os Correios são um dos maiores empregadores do Brasil. Logicamente, com a mudança de seu capital para a iniciativa privada, muitos brasileiros serão demitidos. Com certeza, passando os Correios para a iniciativa privada, seis servidores serão massacrados, justamente aqueles que mais têm trabalhado pelos Correios. Por isso, eu me posiciono a favor de uma saída para os Correios que não seja essa que está sendo conduzida hoje pelo Ministro Kassab, que quer entregar para a iniciativa privada, sem mais nem menos, esse grande patrimônio da sociedade brasileira. Muito obrigado, Sr. Presidente, por esta oportunidade de falar aqui."

<ipython-input-56-7ac2f1896d02>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.clean_discourse[12220] = "Obrigado, Deputado Zé Geraldo, que preside esta Comissão Geral sobre os Correios e Telégrafos. Sras. e Srs. Deputados, convidados, Sr. Presidente dos Correios, demais representantes desta instituição, as dificuldades operacionais dos Correios no momento arrastam para a lama essa instituição que já esteve no topo como uma das mais confiáveis e respeitadas do nosso País. Nos últimos 10 anos, nós temos observado um processo de deterioração dessa importante instituição, um dos maiores patrimônios da sociedade brasileira. Sem dúvida nenhuma, o que ocorreu com os Correios não foi outra coisa senão a má gestão e os assaltos acontecidos na empresa. Isso já foi devidamente comprovado nas investigações 

We can see we still have null entries but we can come up with a solution for them:

In [ ]:
null_speeches_1 = df1.loc[df1.clean_discourse.isna()]
print("There are %i null speeches" % len(null_speeches_1))

There are 40 null speeches


In [ ]:
df1[df1.clean_discourse.isna()][:5]

,date,session,phase,discourse_link,speaker,party,coalition,state,region,original_discourse,clean_discourse
12491,2017-09-27,278.3.55.O,opening,https://www.camara.leg.br/internet/sitaqweb/Te...,carlos manato,solidariedade,0,es,southeast,Ata da 278ª (ducentésima septuagésima oitava) ...,None
12570,2017-11-21,349.3.55.O,general committee,https://www.camara.leg.br/internet/sitaqweb/Te...,edmilson rodrigues,psol,0,pa,north,O SR. DEPUTADO EDMILSON RODRIGUES - Esta sessã...,None
12739,2018-04-09,203.2018,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,prof. gedeão amorim,mdb,1,am,north,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,None
12865,2018-08-08,192.2018,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,stefano aguiar,psd,1,mg,southeast,DISCURSO ENCAMINHADO À MESA PARA PUBLICAÇÃO. D...,None
12894,2018-10-16,216.2018,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,leonardo monteiro,pt,0,mg,southeast,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,None


In [ ]:
# let's create a copy for the df1
df2 = df1.copy()
print(df2.shape[0])

110


Thus, we remove the empty speeches and follow a third approach to the copied dataset.

In [ ]:
df1.dropna(inplace=True)

In [ ]:
df2 = df2.loc[null_speeches_1.index]

In [ ]:
df2.head()

,date,session,phase,discourse_link,speaker,party,coalition,state,region,original_discourse,clean_discourse
12491,2017-09-27,278.3.55.O,opening,https://www.camara.leg.br/internet/sitaqweb/Te...,carlos manato,solidariedade,0,es,southeast,Ata da 278ª (ducentésima septuagésima oitava) ...,None
12570,2017-11-21,349.3.55.O,general committee,https://www.camara.leg.br/internet/sitaqweb/Te...,edmilson rodrigues,psol,0,pa,north,O SR. DEPUTADO EDMILSON RODRIGUES - Esta sessã...,None
12739,2018-04-09,203.2018,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,prof. gedeão amorim,mdb,1,am,north,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,None
12865,2018-08-08,192.2018,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,stefano aguiar,psd,1,mg,southeast,DISCURSO ENCAMINHADO À MESA PARA PUBLICAÇÃO. D...,None
12894,2018-10-16,216.2018,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,leonardo monteiro,pt,0,mg,southeast,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,None


**3rd approach:** we split the data using the pattern "e Srs. Deputados," and retrieve the last string obtained in the split:

In [ ]:
df2['clean_discourse'] = df2.original_discourse.str.split("e Srs. Deputados,", n=1, expand=True)[1]

There's a few null entries, we'll fill then manually.


In [ ]:
df2[df2.clean_discourse.isna()]

,date,session,phase,discourse_link,speaker,party,coalition,state,region,original_discourse,clean_discourse
12491,2017-09-27,278.3.55.O,opening,https://www.camara.leg.br/internet/sitaqweb/Te...,carlos manato,solidariedade,0,es,southeast,Ata da 278ª (ducentésima septuagésima oitava) ...,None
12570,2017-11-21,349.3.55.O,general committee,https://www.camara.leg.br/internet/sitaqweb/Te...,edmilson rodrigues,psol,0,pa,north,O SR. DEPUTADO EDMILSON RODRIGUES - Esta sessã...,None
12894,2018-10-16,216.2018,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,leonardo monteiro,pt,0,mg,southeast,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,None
13110,2019-04-09,255.2019,general committee,https://www.camara.leg.br/internet/sitaqweb/Te...,rodrigo maia,dem,0,rj,southeast,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,None
13714,2019-11-09,268.2019,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,acácio favacho,pros,0,ap,north,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,None
13717,2019-11-09,267.2019,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,josé nelto,pode,0,go,midwest,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,None
13840,2020-02-18,17.202,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,capitão alberto neto,republicanos,0,am,north,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,None
14428,2021-09-09,101.2021,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,wolney queiroz,pdt,0,pe,northeast,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,None
14513,2021-10-14,121.2021,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,capitão alberto neto,republicanos,0,am,north,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,None
14541,2021-10-28,128.2021,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,helder salomão,pt,0,es,southeast,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,None


In [ ]:
null_speeches_2 = df2.loc[df2.clean_discourse.isna()]
print("There are %i null speeches" % len(null_speeches_2))

There are 0 null speeches


We can merge the data frame again.

In [ ]:
df1 = df1.append(df2)
print(df1.shape[0])

110


In [ ]:
df = df.append(df1)
print(df.shape[0])

14602


In [ ]:
df

,date,session,phase,discourse_link,speaker,party,coalition,state,region,original_discourse,clean_discourse
0,2000-01-11,195.2.51.O,short address,https://www.camara.leg.br/internet/SitaqWeb/Te...,nilton capixaba,ptb,0,ro,north,O SR. NILTON CAPIXABA (PTB-RO. Pronuncia o seg...,"Sr. Presidente, Sras. e Srs. Deputados, na se..."
1,2000-01-11,195.2.51.O,parliamentarian communications,https://www.camara.leg.br/internet/SitaqWeb/Te...,sérgio novais,psb,0,ce,northeast,O SR. SÉRGIO NOVAIS (Bloco/PSB-CE. Como Líder....,"Sr. Presidente, Sras. e Srs. Deputados, prime..."
2,2000-01-11,195.2.51.O,short address,https://www.camara.leg.br/internet/SitaqWeb/Te...,marcos afonso,pt,0,ac,north,O SR. MARCOS AFONSO (PT-AC. Pronuncia o seguin...,"Sr. Presidente, Sras. e Srs. Deputados, venho..."
3,2000-01-11,195.2.51.O,short address,https://www.camara.leg.br/internet/SitaqWeb/Te...,expedito júnior,dem,1,ro,north,O SR . EXPEDITO JÚNIOR (PFL-RO. Pronuncia o se...,"Sr. Presidente, Sras. e Srs. Deputados, pode ..."
4,2000-03-11,196.2.51.O,long address,https://www.camara.leg.br/internet/SitaqWeb/Te...,silas câmara,ptb,0,am,north,O SR. SILAS CÂMARA (PTB-AM. Sem revisão do ora...,"Sr. Presidente, Sras. e Srs. Deputados, é com..."
...,...,...,...,...,...,...,...,...,...,...,...
14435,2021-09-14,103.2021,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,leo de brito,pt,0,ac,north,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,"em primeiro lugar, quero me solidarizar com o..."
14513,2021-10-14,121.2021,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,capitão alberto neto,republicanos,0,am,north,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,"Exmo. Sr. Presidente, Sras. Deputadas, Srs. De..."
14541,2021-10-28,128.2021,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,helder salomão,pt,0,es,southeast,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,"Sr. Presidente, Sras. e Srs. Deputadas e Deput..."
14555,2021-11-16,136.2021,closure,https://www.camara.leg.br/internet/sitaqweb/Te...,nilto tatto,pt,0,sp,southeast,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUT...,"Sr. Presidente, registro palestra proferida pe..."


In [ ]:
# saving as a brand new clean Amazon discourses data set
df.to_csv('amazon_clean_discourses.tsv', sep='\t', encoding='utf-8', index=False)